In [1]:
import networkx as nx
from pickle import *
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import yaml
from datascience import *

In [2]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

### Congress on Twitter data, version 2

The original Congress on Twitter data was difficult to link to biographical information about each legislator.

In order to address this issue, I am releasing this second version of the Congress on Twitter dataset. I recommend that you use this dataset instead of the previous one, but you are free to choose either one.

The main differences between this dataset and the previous one are:

1. This dataset only consists of 'official' Twitter accounts for each legislator (many legislators also have distinct campaign-related accounts)
2. This dataset has been linked to the biographical information about each legislator, so it has information like party, state, et cetera

In order to narrow the data down to official Twitter accounts, and to merge it with the biographical information, I had to omit

* 16 legislators for whom I did not have official Twitter account info
* 3 legislators whose official Twitter accounts didn't follow any other Members of Congress (so they were isolates in the network)

#### Reading the data in

In [3]:
official_congress_twitter, official_congress_friends, legislator_data = load(open('../congress_twitter_v2.pickle', 'rb'))
legislator_data = Table().from_df(legislator_data)

In [4]:
congress_members = load(open('../../project/congress_twitter/data/congress_list_members.pickle', 'rb'))

In [5]:
congress_members_data = pd.DataFrame([x for x in congress_members])
congress_members_data = Table().from_df(congress_members_data)

In [43]:
congress_members_data

contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,has_extended_profile,id,id_str,is_translation_enabled,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,status,statuses_count,time_zone,url,utc_offset,verified,mod_screen_name
False,Tue Jun 28 14:57:49 +0000 2016,True,False,Representing MA's 7th District in the U.S. House of Repr ...,{'description': {'urls': [{'expanded_url': 'https://www. ...,9,False,1621,False,94,False,False,747806211983179776,747806211983179776,False,False,en,45,"Somerville, MA",Rep. Michael Capuano,False,F5F8FA,None,None,False,https://pbs.twimg.com/profile_banners/747806211983179776 ...,http://pbs.twimg.com/profile_images/747855038681325569/N ...,https://pbs.twimg.com/profile_images/747855038681325569/ ...,2B7BB9,C0DEED,DDEEF6,333333,True,False,RepMikeCapuano,{'text': 'Read about the conclusion of the district work ...,35,Eastern Time (US & Canada),https://t.co/3ljoXJRbEf,-14400,True,repmikecapuano
False,Thu Feb 19 20:08:54 +0000 2015,True,False,Representing California's 33rd District in Congress - On ...,"{'description': {'urls': []}, 'url': {'urls': [{'expande ...",100,False,6692,False,4690,True,False,3044993235,3044993235,False,False,en,300,Washington D.C. & Los Angeles,Rep. Ted Lieu,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/3044993235/1461336687,http://pbs.twimg.com/profile_images/756518703517360129/z ...,https://pbs.twimg.com/profile_images/756518703517360129/ ...,0084B4,C0DEED,DDEEF6,333333,True,False,RepTedLieu,{'text': 'RT @MayorOfLA: 5th year running that LA has be ...,1924,Eastern Time (US & Canada),https://t.co/SFF1Hjj9ww,-14400,True,reptedlieu
False,Mon Feb 09 16:00:36 +0000 2015,False,False,Congresswoman Representing American Samoa,"{'description': {'urls': []}, 'url': {'urls': [{'expande ...",0,False,494,False,0,False,False,3026622545,3026622545,False,False,en,52,"Washington, D.C.",Congresswoman Amata,False,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/3026622545/1423503404,http://pbs.twimg.com/profile_images/564816554185859073/f ...,https://pbs.twimg.com/profile_images/564816554185859073/ ...,94D487,000000,000000,000000,False,False,RepAmata,"{'text': ""Hello everyone, I'm excited to join Twitter to ...",1,None,http://t.co/L7B6x0yzTS,nan,True,repamata
False,Wed Feb 04 21:01:23 +0000 2015,True,False,Pete Aguilar represents CA’s 31st Congressional District ...,"{'description': {'urls': []}, 'url': {'urls': [{'expande ...",1,False,2894,False,143,False,False,3018670151,3018670151,False,False,en,218,"San Bernardino, CA",Pete Aguilar,False,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,nan,http://pbs.twimg.com/profile_images/724955126897725440/B ...,https://pbs.twimg.com/profile_images/724955126897725440/ ...,0084B4,C0DEED,DDEEF6,333333,True,False,RepPeteAguilar,"{'text': ""It's #BreastCancerAwarenessMonth. Learn about ...",731,None,http://t.co/o4deRoUnuK,nan,True,reppeteaguilar
False,Tue Jan 13 15:08:48 +0000 2015,False,False,U.S. Congressman (WI-06),"{'description': {'urls': []}, 'url': {'urls': [{'expande ...",13,False,1752,False,205,False,False,2976606250,2976606250,False,False,en,148,"Washington, D.C.",Rep. Glenn Grothman,False,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile

#### Network of official legislator Twitter accounts

In [7]:
official_congress_twitter.number_of_nodes()

519

In [36]:
#measures how many people they are friends with
avg_degree = 2*official_congress_twitter.number_of_edges()\
        /official_congress_twitter.number_of_nodes()
avg_degree

184.03853564547205

In [9]:
#measures how likely nodes are to cluster together and 
#form very tight bonds
nx.average_clustering(official_congress_twitter)

0.6698933484362136

In [13]:
#measures similarity of connections in the graph with 
#respect to the given attribute (party)
nx.attribute_assortativity_coefficient(
    official_congress_twitter, 'party')

0.5984762733850006

In [42]:
#calculate transitivity
nx.transitivity(official_congress_twitter)

0.5901175440115135

#### Legislator biographical Info

In [46]:
legislator_data

twitter,first,last,middle,nickname,official_full,suffix,address,caucus,class,contact_form,district,end,fax,office,party,phone,rss_url,start,state,state_rank,type,url,birthday,gender,religion,twitter_id,bioguide
sensherrodbrown,Sherrod,Brown,nan,nan,Sherrod Brown,nan,713 Hart Senate Office Building Washington DC 20510,nan,1,http://www.brown.senate.gov/contact,nan,2019-01-03,202-228-6321,713 Hart Senate Office Building,Democrat,202-224-2315,http://www.brown.senate.gov/rss/feeds/?type=all&,2013-01-03,OH,senior,sen,http://www.brown.senate.gov,1952-11-09,M,Lutheran,4.39108e+07,B000944
senatorcantwell,Maria,Cantwell,nan,nan,Maria Cantwell,nan,511 Hart Senate Office Building Washington DC 20510,nan,1,http://www.cantwell.senate.gov/public/index.cfm/email-maria,nan,2019-01-03,202-228-0514,511 Hart Senate Office Building,Democrat,202-224-3441,http://www.cantwell.senate.gov/public/index.cfm/rss/feed,2013-01-03,WA,junior,sen,https://www.cantwell.senate.gov,1958-10-13,F,Roman Catholic,1.17502e+08,C000127
senatorcardin,Benjamin,Cardin,L.,nan,Benjamin L. Cardin,nan,509 Hart Senate Office Building Washington DC 20510,nan,1,http://www.cardin.senate.gov/contact/,nan,2019-01-03,202-224-1651,509 Hart Senate Office Building,Democrat,202-224-4524,http://www.cardin.senate.gov/rss/feeds/?type=all,2013-01-03,MD,junior,sen,https://www.cardin.senate.gov,1943-10-05,M,Jewish,1.09071e+08,C000141
senatorcarper,Thomas,Carper,Richard,nan,Thomas R. Carper,nan,513 Hart Senate Office Building Washington DC 20510,nan,1,http://www.carper.senate.gov/public/index.cfm/email-sena ...,nan,2019-01-03,202-228-2190,513 Hart Senate Office Building,Democrat,202-224-2441,http://www.carper.senate.gov/public/index.cfm/rss/feed,2013-01-03,DE,senior,sen,http://www.carper.senate.gov/public,1947-01-23,M,Presbyterian,2.49788e+08,C000174
senbobcasey,Robert,Casey,P.,Bob,"Robert P. Casey, Jr.",Jr.,393 Russell Senate Office Building Washington DC 20510,nan,1,http://www.casey.senate.gov/contact/,nan,2019-01-03,202-228-0604,393 Russell Senate Office Building,Democrat,202-224-6324,http://www.casey.senate.gov/rss/feeds/?all,2013-01-03,PA,senior,sen,https://www.casey.senate.gov,1960-04-13,M,nan,1.71599e+08,C001070
senbobcorker,Bob,Corker,nan,nan,Bob Corker,nan,425 Dirksen Senate Office Building Washington DC 20510,nan,1,http://www.corker.senate.gov/public/index.cfm/emailme,nan,2019-01-03,202-228-0566,425 Dirksen Senate Office Building,Republican,202-224-3344,http://www.corker.senate.gov/public/index.cfm/rss/feed,2013-01-03,TN,junior,sen,http://www.corker.senate.gov,1952-08-24,M,nan,7.33038e+07,C001071
senfeinstein,Dianne,Feinstein,nan,nan,Dianne Feinstein,nan,331 Hart Senate Office Building Washington DC 20510,nan,1,https://www.feinstein.senate.gov/public/index.cfm/e-mail-me,nan,2019-01-03,202-228-3954,331 Hart Senate Office Building,Democrat,202-224-3841,http://www.feinstein.senate.gov/public/?a=rss.feed,2013-01-03,CA,senior,sen,http://www.feinstein.senate.gov,1933-06-22,F,Jewish,4.76257e+08,F000062
senorrinhatch,Orrin,Hatch,G.,nan,Orrin G. Hatch,nan,104 Hart Senate Office Building Washington DC 20510,nan,1,http://www.hatch.senate.gov/public/index.cfm/contact?p=E ...,nan,2019-01-03,202-224-6331,104 Hart Senate Office Building,Republican,202-224-5251,http://www.hatch.senate.gov/public/index.cfm/rss/feed,2013-01-03,UT,senior,sen,http://www.hatch.senate.gov,1934-03-22,M,Latter Day Saints,2.62757e+08,H000338
nan,Amy,Klobuchar,Jean,nan,Amy Klobuchar,nan,302 Hart Senate Office Building Washington DC 20510,nan,1,http://www.klobuchar.senate.gov/public/email-amy,nan,2019-01-03,202-228-2186,302 Hart Senate Office Building,Democrat,202-224-3244,nan,2013-01-03,MN,senior,sen,http://www.klobuchar.senate.gov,1960-05-25,F,nan,nan,K000367
mccaskilloffice,Claire,McCaskill,nan,nan,Claire McCaskill,nan,730 Hart Senate Office Building Washington DC 20510,nan,1,http://www.mccaskill.senate.gov/?p=contact,nan,2019-01-03,202-228-6326,730 Hart Senate Office Building,Democrat,202-224-6154,http://mccaskill.senate.gov

#### Joining the Tables

In [37]:
desired_col = congress_members_data.column("screen_name")
congress_members_data = congress_members_data.with_columns(
    "mod_screen_name", [x.lower() for x in desired_col])
joined_tables = legislator_data.join(
    "twitter", congress_members_data, "mod_screen_name")
joined_tables = joined_tables.drop("screen_name")
#joined_tables

#### Analyzing using official_congress_twitter (directed, unweighted graph)

In [18]:
#Republicans have majority in Congress 
group_by_party = joined_tables.group("party")
group_by_party

party,count
Democrat,211
Independent,2
Republican,282


In [19]:
#Majority party has more followers
num_followers = joined_tables.select(
    "party", "followers_count").group("party", np.sum)
num_followers.show()

party,followers_count sum
Democrat,8305302
Independent,2681450
Republican,9013929


In [20]:
#See relationship between average degree and party
id_and_degree = Table().with_columns(
    "Twitter ID", official_congress_twitter.nodes(),
    "Degree", official_congress_twitter.degree().values())

id_and_party = joined_tables.select("twitter_id", "party")
i_p_d = id_and_party.join("twitter_id", id_and_degree, \
            "Twitter ID").sort("Degree", descending=True)

dem_only = i_p_d.drop(
    "twitter_id").where("party", are.equal_to("Democrat"))
rep_only = i_p_d.drop(
    "twitter_id").where("party", are.equal_to("Republican"))

avg_degree_dem = np.sum(dem_only.column("Degree"))/dem_only.num_rows
print("Average Degree for Democrats: ", avg_degree_dem)
avg_degree_rep = np.sum(rep_only.column("Degree"))/rep_only.num_rows
print("Average Degree for Republicans: ", avg_degree_rep)

Average Degree for Democrats:  184.578199052
Average Degree for Republicans:  186.067375887


In [21]:
#See relationship between # of twitter statuses and party
p_s = joined_tables.select("party", "statuses_count")
p_s_count = p_s.group("party")

p_s = p_s.group("party", np.sum).relabel(
            "statuses_count sum", "statuses_count")
avg_status_count = p_s.column(
    "statuses_count")/p_s_count.column("count")

print("Average Status Count for Democrats: ", \
      avg_status_count.item(0))
print("Average Status Count for Republicans: ", \
      avg_status_count.item(2))

Average Status Count for Democrats:  3557.2417061611372
Average Status Count for Republicans:  3030.294326241135


In [22]:
#See relationship between # of Twitter friends and party
p_f = joined_tables.select("party", "friends_count")
p_f_count = p_f.group("party")

p_f = p_f.group("party", np.sum).relabel(
    "friends_count sum", "friends_count")
avg_status_count = p_f.column(
    "friends_count")/p_f_count.column("count")

print("Average Friend Count for Democrats: ", avg_status_count.item(0))
print("Average Friend Count for Republicans: ", avg_status_count.item(2))

Average Friend Count for Democrats:  2489.872037914692
Average Friend Count for Republicans:  1568.1418439716313


In [38]:
#list of twitter id of members of each party
dem = [n for n in official_congress_twitter if \
    official_congress_twitter.node[n]['party'] == 'Democrat']
rep = [n for n in official_congress_twitter if \
    official_congress_twitter.node[n]['party'] == 'Republican']

#creates respective subgraphs for each party
dem_graph = official_congress_twitter.subgraph(dem)
rep_graph = official_congress_twitter.subgraph(rep)

In [39]:
#Calculates degree of average neighbor
dict_avg_neighbor_dem = nx.average_neighbor_degree(dem_graph)
dict_avg_neighbor_rep = nx.average_neighbor_degree(rep_graph)

avg_neighbor_degree_dem = sum(
    dict_avg_neighbor_dem.values())/len(dict_avg_neighbor_dem)
avg_neighbor_degree_rep = sum(
    dict_avg_neighbor_rep.values())/len(dict_avg_neighbor_rep)

print("Average neighbor degree for Democrats: ", avg_neighbor_degree_dem)
print("Average neighbor degree for Republicans: ", avg_neighbor_degree_rep)

Average neighbor degree for Democrats:  152.9167886240047
Average neighbor degree for Republicans:  185.25202473522882


In [25]:
#calculates clustering coefficient for each party
#Measures extent of transitivity 
dem_clustering = nx.average_clustering(dem_graph)
rep_clustering = nx.average_clustering(rep_graph)

print("Average clustering coefficient for Democrats: ", dem_clustering)
print("Average clustering coefficient for Republicans: ", rep_clustering)

Average clustering coefficient for Democrats:  0.7786453266184385
Average clustering coefficient for Republicans:  0.753744656943668


In [26]:
#Calculates transitivity for each party
#Indicates the tendency for a person's friends to be friends with each other
dem_transitivity = nx.transitivity(dem_graph)
rep_transitivity = nx.transitivity(rep_graph)

print("transitivity for Democrats: ", dem_transitivity)
print("transitivity coefficient for Republicans: ", rep_transitivity)

transitivity for Democrats:  0.7267131755563405
transitivity coefficient for Republicans:  0.7055725494652744


In [27]:
#Calculates assortativity coefficient for each party
#if assortativity coefficient > 1, that means members of the same group 
#are more likely to be connected to each other than to other groups
nx.attribute_assortativity_coefficient(official_congress_twitter, "party")

0.5984762733850006

#### Analyzing using weighted_UD_graph (weighted, undirected version of official_congress_twitter)

In [28]:
#create undirected, weighted version of official_congress_twitter
#if they follow each other mutually, weight = 2
#if it's only one-way, weight = 1
weighted_UD_graph = nx.Graph()

for key in official_congress_friends.keys():
    values = official_congress_friends.get(key)
    for val in values:
        other_val = official_congress_friends.get(val)
        if key in other_val:
            weighted_UD_graph.add_edge(key, val, weight=2)
        else:
            weighted_UD_graph.add_edge(key, val, weight=1)

In [40]:
#See relationship between average degree and party
id_and_degree = Table().with_columns(
    "Twitter ID", weighted_UD_graph.nodes(),
    "Degree", weighted_UD_graph.degree().values())

id_and_party = joined_tables.select("twitter_id", "party")
i_p_d = id_and_party.join("twitter_id", id_and_degree, \
                    "Twitter ID").sort("Degree", descending=True)

dem_only = i_p_d.drop("twitter_id").where("party", \
                                are.equal_to("Democrat"))
rep_only = i_p_d.drop("twitter_id").where("party", \
                                are.equal_to("Republican"))

avg_degree_dem = np.sum(dem_only.column(
        "Degree"))/dem_only.num_rows
print("Average Degree for Democrats: ", avg_degree_dem)
avg_degree_rep = np.sum(rep_only.column(
        "Degree"))/rep_only.num_rows
print("Average Degree for Republicans: ", avg_degree_rep)

Average Degree for Democrats:  190.09478673
Average Degree for Republicans:  179.191489362
